In [1]:
import requests
import json
import redis
import pandas as pd
import os
from urllib.parse import urlencode

In [2]:
# caching
redis_client = redis.Redis(host = 'localhost', port = 6379, db = 0)

In [3]:
api_key = # api_key

In [4]:
def fetch_place(address, update:bool = False):
    """
    takes in an address and get the json data of the place. If not found in cache then would 
    call the google map API to fetch data.
    """
    
    if address[:3] != "宜蘭縣":
        address = "宜蘭縣" + address
    
    place_key = f"{address}_place"
    place = redis_client.get(place_key)
    
    if update:
        place = None
    
    if not place:
        print('Could not find place in cache. Retrieving from Google Maps API...')
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/json"
        params = {"address": address, "key": api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            place = []
        else:
            place = r.json()['results']
        
        redis_client.set(place_key, json.dumps(place))
    
    else:
        print('Found place in cache, serving from redis...')
        place = json.loads(place)
        
    return place

In [5]:
def extract_lat_lng(address, update=False):
    
    place = fetch_place(address, update)
    if place:
        place = place[0]
        location = place['geometry']['location']
        lat, lng = location['lat'], location['lng']
    else:
        lat, lng = None, None
    
    return lat, lng

In [7]:
# === import csv === #
df = pd.read_csv('11Input/02DataProcessed/sample.csv')
df.head()

,Thevillagesandtownsurbandis,transactionsign,landsectorpositionbuildingse,landshiftingtotalareasquare,theusezoningorcompilesandc,thenonmetropolislandusedist,nonmetropolislanduse,transactionyearmonthandday,transactionpennumber,shiftinglevel,...,serialnumber,mainbuildingarea,auxiliarybuildingarea,balconyarea,elevator,transactionnumber,year,month,insample,notinsample
0,宜蘭市,房地(土地+建物),宜蘭縣宜蘭市農權路三段２７巷７號,131.28,都市：其他:住宅區,NaN,NaN,1111104,土地1建物1車位0,全,...,RPPOMLSKJIHGFBG97DA,265.71,0.00,24.04,有,NaN,2022,11,1,0
1,宜蘭市,房地(土地+建物),宜蘭縣宜蘭市同興街１４６號,36.53,都市：其他:商業區,NaN,NaN,1111101,土地1建物1車位0,全,...,RPPPMLLLJIHGFBG47DA,72.23,0.00,0.00,無,NaN,2022,11,1,0
2,員山鄉,房地(土地+建物),宜蘭縣員山鄉大安路９２之２８號,115.66,NaN,特定農業區,甲種建築用地,1111105,土地1建物1車位0,全,...,RPWNMLNLJIHGFBG18DA,176.43,3.97,10.13,無,NaN,2022,11,1,0
3,三星鄉,房地(土地+建物),宜蘭縣三星鄉健富路二段３４６巷８號,89.00,NaN,特定農業區,甲種建築用地,1111102,土地2建物1車位0,全,...,RPUNMLOLJIHGFAG97DA,145.77,0.00,14.56,無,NaN,2022,11,1,0
4,員山鄉,房地(土地+建物),宜蘭縣員山鄉員山路一段３６０號,101.15,都市：其他:住宅區,NaN,NaN,1111115,土地2建物1車位0,全,...,RPRNMLSLJIHGFBG18DA,189.99,0.00,13.52,有,NaN,2022,11,1,0


In [8]:
df.shape

(8993, 37)

In [9]:
addresses = df['landsectorpositionbuildingse'].to_numpy()
locations = [None] * len(addresses)
for i in range(len(locations)):
    lat, lng = extract_lat_lng(addresses[i], update=False)
    locations[i] = [lat, lng]

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

...

Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not 

Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not 

Could not find place in cache. Retrieving from Google Maps API...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Could not find place in cache. Retrieving from Google Maps API...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving from redis...
Found place in cache, serving fr

In [10]:
df_locations = pd.DataFrame(locations, columns=['lat', 'lng'])
df_withloc = pd.concat([df, df_locations], axis=1)
df_withloc

,Thevillagesandtownsurbandis,transactionsign,landsectorpositionbuildingse,landshiftingtotalareasquare,theusezoningorcompilesandc,thenonmetropolislandusedist,nonmetropolislanduse,transactionyearmonthandday,transactionpennumber,shiftinglevel,...,auxiliarybuildingarea,balconyarea,elevator,transactionnumber,year,month,insample,notinsample,lat,lng
0,宜蘭市,房地(土地+建物),宜蘭縣宜蘭市農權路三段２７巷７號,131.28,都市：其他:住宅區,NaN,NaN,1111104,土地1建物1車位0,全,...,0.00,24.04,有,NaN,2022,11,1,0,24.748647,121.742330
1,宜蘭市,房地(土地+建物),宜蘭縣宜蘭市同興街１４６號,36.53,都市：其他:商業區,NaN,NaN,1111101,土地1建物1車位0,全,...,0.00,0.00,無,NaN,2022,11,1,0,24.763950,121.754159
2,員山鄉,房地(土地+建物),宜蘭縣員山鄉大安路９２之２８號,115.66,NaN,特定農業區,甲種建築用地,1111105,土地1建物1車位0,全,...,3.97,10.13,無,NaN,2022,11,1,0,24.736621,121.698002
3,三星鄉,房地(土地+建物),宜蘭縣三星鄉健富路二段３４６巷８號,89.00,NaN,特定農業區,甲種建築用地,1111102,土地2建物1車位0,全,...,0.00,14.56,無,NaN,2022,11,1,0,24.673226,121.688242
4,員山鄉,房地(土地+建物),宜蘭縣員山鄉員山路一段３６０號,101.15,都市：其他:住宅區,NaN,NaN,1111115,土地2建物1車位0,全,...,0.00,13.52,有,NaN,2022,11,1,0,24.742365,121.722595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8988,員山鄉,房地(土地+建物),宜蘭縣員山鄉榮光路２６４之１號,82.00,NaN,鄉村區,乙種建築用地,1070129,土地1建物1車位0,全,...,0.00,0.00,NaN,NaN,2018,1,1,0,24.720355,121.690272
8989,頭城鎮,房地(土地+建物),宜蘭縣頭城鎮新興路231-30號,100.32,住,NaN,NaN,1070104,土地2建物1車位0,全,...,0.00,0.00,NaN,NaN,2018,1,1,0,24.854977,121.819696
8990,五結鄉,房地(土地+建物),宜蘭縣五結鄉仁五路一段５１巷２號,158.32,農,NaN,NaN,1070123,土地3建物1車位0,全,...,0.00,15.64,NaN,NaN,2018,1,1,0,24.685596,121.786217
8991,冬山鄉,房地(土地+建物),宜蘭縣冬山鄉寶和路１７６巷１２號,92.95,NaN,特定農業區,甲種建築用地,1070107,土地0建物1車位0,全,...,0.00,14.81,NaN,NaN,2018,1,1,0,24.633176,121.769090


In [11]:
sum(df_withloc['lat'].isnull()) # 0; No missing latitudes

0

In [29]:
# == checking correctness of lat and lng == #

county_place = fetch_place('宜蘭縣')[0]
b_north = county_place['geometry']['bounds']['northeast']['lat']
b_east = county_place['geometry']['bounds']['northeast']['lng']
b_south = county_place['geometry']['bounds']['southwest']['lat']
b_west = county_place['geometry']['bounds']['southwest']['lng']

def in_bounds(row):
    if row['lat'] > b_north or row['lat'] < b_south:
        return False
    if row['lng'] > b_east or row['lng'] < b_west:
        return False
    return True

df_withloc['in_b'] = df_withloc.apply(lambda row: in_bounds(row), axis=1)
df.shape[0] - sum(df_withloc['in_b']) # 0; No places out of boundary

Found place in cache, serving from redis...


0

In [26]:
df_withloc.to_csv("11Input/02DataProcessed/geocoded_sample.csv")